---
# Grouping and Aggregating Data
Up until this point, we have been building our pandas foundations and mainly been doing the technical parts (the operations). Now we will be using what we've learned to analyze and explore our stackoverflow data. 

---

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display


In [2]:
# Function for printing a horizontal line. For display purpose
def printhr(s: str = None, n: int = 40):
    """Print a horizontal rule of the character "=" of length n.

    Args:
        s (str, optional): Header message. Defaults to None.
        n (int, optional): Number of characters. Defaults to 50.
    """

    if s:
        print("=" * int(n / 2), s, "=" * int(n / 2))
    else:
        print("=" * n)


In [3]:
# Stackoverflow developer survey
df = pd.read_csv("data/survey_results_public_2022.csv", index_col="ResponseId")
schema_df = pd.read_csv("data/survey_results_schema.csv", index_col="qname")


---
## Aggregating Data
Aggregation is any process where data is expressed in summary form. (e.g. taking the mean of data)

---

---
### `describe()` Method
Pandas has a `describe()` method that shows different aggregate statistics of the column(s) (determined by the **include** parameter. Defaults to numeric columns).

---

In [4]:
# Applied on df
display(df.describe())

# Applied on a Series (single column)
display(df["ConvertedCompYearly"].describe())


,CompTotal,VCHostingPersonal use,VCHostingProfessional use,WorkExp,ConvertedCompYearly
count,3.842200e+04,0.0,0.0,36769.000000,3.807100e+04
mean,2.342434e+52,NaN,NaN,10.242378,1.707613e+05
std,4.591478e+54,NaN,NaN,8.706850,7.814132e+05
min,0.000000e+00,NaN,NaN,0.000000,1.000000e+00
25%,3.000000e+04,NaN,NaN,4.000000,3.583200e+04
50%,7.750000e+04,NaN,NaN,8.000000,6.784500e+04
75%,1.540000e+05,NaN,NaN,15.000000,1.200000e+05
max,9.000000e+56,NaN,NaN,50.000000,5.000000e+07


count    3.807100e+04
mean     1.707613e+05
std      7.814132e+05
min      1.000000e+00
25%      3.583200e+04
50%      6.784500e+04
75%      1.200000e+05
max      5.000000e+07
Name: ConvertedCompYearly, dtype: float64

---
### `value_counts()` Method
This method counts the unique row values of a Series or a DataFrame. When used on a DataFrame, a list of labels can be passed to count the unique combinations of the passed labels there are.  
**normalize** is a bool parameter which determines whether the counts are showed in frequencies, or in proportion.

---

In [5]:
# Used on a df with combination of columns
series = df.value_counts(["ConvertedCompYearly", "RemoteWork"])
display(series)


ConvertedCompYearly  RemoteWork                          
150000.0             Fully remote                            270
200000.0             Fully remote                            242
120000.0             Fully remote                            208
100000.0             Fully remote                            174
180000.0             Fully remote                            172
                                                            ... 
43428.0              Fully remote                              1
43380.0              Hybrid (some remote, some in-person)      1
43356.0              Hybrid (some remote, some in-person)      1
43346.0              Fully remote                              1
50000000.0           Full in-person                            1
Name: count, Length: 11465, dtype: int64

In [6]:
# Used on a Series and using the normalize parameter
series = df["ConvertedCompYearly"].value_counts(normalize=True)
display(series)

# We can multiply all the values in the Series by 100
# to show the values in their percentage (out of 100%)
display(series * 100)

ConvertedCompYearly
150000.0    0.010323
200000.0    0.009509
120000.0    0.008957
63986.0     0.007985
100000.0    0.007328
              ...   
76472.0     0.000026
1368.0      0.000026
104952.0    0.000026
3648.0      0.000026
110245.0    0.000026
Name: proportion, Length: 7909, dtype: float64

ConvertedCompYearly
150000.0    1.032282
200000.0    0.950855
120000.0    0.895695
63986.0     0.798508
100000.0    0.732841
              ...   
76472.0     0.002627
1368.0      0.002627
104952.0    0.002627
3648.0      0.002627
110245.0    0.002627
Name: proportion, Length: 7909, dtype: float64

---
### Measures of Central Tendency
Pandas has a built-in `mean()`, `median()`, and `mode()` methods. These methods can be applied to both Series and DataFrames. 

To apply the methods on all numeric columns of the DataFrame, **True** should be passed to **numeric_only**

---

In [7]:
# Taking median of all numeric columns
medians = df.median(numeric_only=True)
display(medians)

# Note that CompTotal is not normalized; currencies differ


CompTotal                    77500.0
VCHostingPersonal use            NaN
VCHostingProfessional use        NaN
WorkExp                          8.0
ConvertedCompYearly          67845.0
dtype: float64

---
We might ask: what is the typical compensation of software developers in 2022?  
If we decide to use the median as a measure:

**Note: all examples from hereon out are for illustrative purposes only. The methods used might not be the best way to answer the questions**  

---

In [8]:
# Taking the median compensation
# We will use ConvertedCompYearly as it normalizes everything to USD
median_comp = df["ConvertedCompYearly"].median()
display(median_comp)


67845.0

---
We can see that the median salary for software developers is $67,845. (This of course lacks context and does not take account other factors such as experience, among others)

---

---
## Grouping Data
pandas has a group object that can be created using the `groupby()` method. This method returns a groupby object. There are 2 groupby objects: the DataFrame.groupby and the Series.groupby, where some methods are exlusive to each.

`groupby()` takes a **by** parameter that will group rows according to the passed value. A list of labels can be passed to group entries with combinations of the labels.  


### groupby.groups Attribute
Groupings of the current groupby object can be viewed by the `.groups` attribute. This returns a dict of the groupings of group \<group_label\>:\<list_of_indexes_of_that_group\> key:value pairs.

---

---
**Suppose we want to know the frequencies of organization size grouped by country**

---

In [9]:
# First we create a DataFrame.groupby object:
country_group = df.groupby("Country")

# View groups
groups = country_group.groups
display(groups)


{'Afghanistan': [1960, 2449, 2738, 3642, 3776, 3887, 4503, 5465, 6243, 6777, 6892, 7945, 8891, 10234, 10911, 13852, 15608, 17039, 17470, 19202, 21766, 22877, 23162, 23224, 24813, 25963, 26618, 28435, 28583, 29705, 30041, 30851, 32133, 33329, 33429, 34044, 38513, 42123, 42871, 43640, 43691, 46026, 46423, 48960, 49274, 51189, 51548, 53961, 55310, 55394, 55666, 56189, 56267, 56356, 58078, 60147, 60517, 60626, 60650, 61618, 61768, 61936, 64907, 67566, 67667, 69400, 69587, 70466, 71716], 'Albania': [5622, 5832, 7569, 7679, 8054, 9519, 9650, 10347, 12748, 14027, 17275, 18462, 19216, 19864, 21056, 21121, 21147, 21212, 23139, 23213, 23694, 24581, 25126, 25179, 29793, 32301, 33423, 33467, 33980, 34050, 34219, 35056, 35598, 36239, 40659, 42517, 42875, 43538, 43559, 44382, 48376, 49362, 52768, 53227, 56883, 57991, 58291, 60812, 63271, 65213, 65421, 66018, 67962, 68078, 68320, 68850, 68973, 72371, 72923], 'Algeria': [1454, 6267, 6714, 11295, 11327, 19841, 19903, 20705, 21828, 23355, 23706, 23858, 

In [10]:
# Check frequency of org size on grouped countries
grouped_freq = country_group["OrgSize"].value_counts()
display(grouped_freq.head(25))


Country      OrgSize                                           
Afghanistan  2 to 9 employees                                       6
             100 to 499 employees                                   5
             10 to 19 employees                                     4
             500 to 999 employees                                   4
             Just me - I am a freelancer, sole proprietor, etc.     4
             I don’t know                                           3
             1,000 to 4,999 employees                               2
             10,000 or more employees                               2
             20 to 99 employees                                     2
             5,000 to 9,999 employees                               2
Albania      10 to 19 employees                                    10
             2 to 9 employees                                       9
             20 to 99 employees                                     7
             100 to 499 em

---
Since value_counts() returns a Series, we can index to a country and see its individual frequency. This is useful as we only need to change the index to get specific values.

---

In [11]:
display(grouped_freq)
printhr()
# Select subsets in grouped frequency

# Germany
germany_freq = grouped_freq.loc["Germany"]
display(germany_freq)

# Multiple countries
multiple_freq = grouped_freq.loc[["Japan", "India", "China"]]
display(multiple_freq)


Country      OrgSize                                           
Afghanistan  2 to 9 employees                                      6
             100 to 499 employees                                  5
             10 to 19 employees                                    4
             500 to 999 employees                                  4
             Just me - I am a freelancer, sole proprietor, etc.    4
                                                                  ..
Zimbabwe     1,000 to 4,999 employees                              2
             2 to 9 employees                                      2
             100 to 499 employees                                  2
             Just me - I am a freelancer, sole proprietor, etc.    2
             10 to 19 employees                                    1
Name: count, Length: 1239, dtype: int64

OrgSize
20 to 99 employees                                    834
100 to 499 employees                                  765
10,000 or more employees                              425
1,000 to 4,999 employees                              399
2 to 9 employees                                      355
10 to 19 employees                                    320
500 to 999 employees                                  303
Just me - I am a freelancer, sole proprietor, etc.    186
5,000 to 9,999 employees                              162
I don’t know                                           33
Name: count, dtype: int64

Country  OrgSize                                           
Japan    20 to 99 employees                                     60
         100 to 499 employees                                   50
         10,000 or more employees                               32
         1,000 to 4,999 employees                               31
         2 to 9 employees                                       27
         10 to 19 employees                                     24
         Just me - I am a freelancer, sole proprietor, etc.     21
         500 to 999 employees                                   15
         5,000 to 9,999 employees                                6
         I don’t know                                            2
India    10,000 or more employees                              952
         20 to 99 employees                                    784
         100 to 499 employees                                  693
         1,000 to 4,999 employees                              424
  

---
### Applying Methods on Group Objects
Most (if not all) methods that can be applied to Series and DataFrames can be applied to Group objects.  

Multiple aggregations can be done by using the `agg()` (or `aggregate()`, they are equivalent) and passing in the a list of function names to be applied.

---

In [12]:
# Get median of yearly salary (USD) of each country
comp_group = country_group["ConvertedCompYearly"]
comp_median = comp_group.median()
display(comp_median)


Country
Afghanistan                             10894.0
Albania                                 15360.0
Algeria                                  7440.0
Andorra                                 76784.0
Angola                                   3073.5
                                         ...   
Venezuela, Bolivarian Republic of...    12000.0
Viet Nam                                11388.0
Yemen                                   15000.0
Zambia                                   6612.0
Zimbabwe                                11400.0
Name: ConvertedCompYearly, Length: 180, dtype: float64

In [13]:
# Get mean, median, and maximum of yearly salary (USD) of each country
comp_group = country_group["ConvertedCompYearly"]
comp_stats = comp_group.agg(["mean", "median", "max"])
display(comp_stats)

# Get same stats for USA
comp_stats_usa = comp_stats.loc["United States of America"]
display(comp_stats_usa)


,mean,median,max
Country,,,
Afghanistan,33914.666667,10894.0,158484.0
Albania,24895.148148,15360.0,106780.0
Algeria,12732.888889,7440.0,31992.0
Andorra,98003.125000,76784.0,213287.0
Angola,6222.750000,3073.5,17304.0
...,...,...,...
"Venezuela, Bolivarian Republic of...",17079.400000,12000.0,74400.0
Viet Nam,37707.563830,11388.0,995064.0
Yemen,17464.666667,15000.0,40000.0


mean      3.849382e+05
median    1.450000e+05
max       3.500000e+07
Name: United States of America, dtype: float64

---
`agg()` works on DataFrames and Series too! 

---

In [14]:
# Since we will use numerical aggregate functions (median and mode),
# let's create a df consisting only of numeric columns
num_df = df.select_dtypes(include="number")

# Drop empty columns (idk why this are even in the survey. They have no entries)
num_df = num_df.drop(["VCHostingPersonal use", "VCHostingProfessional use"], axis=1)

series = num_df.agg(["median", "mean"])
display(series)

# Note that CompTotal is not normalized; currencies differ and you might see
# extreme contrasts between values.

,CompTotal,WorkExp,ConvertedCompYearly
median,7.750000e+04,8.000000,67845.000000
mean,2.342434e+52,10.242378,170761.277849


In [15]:
# agg() on Series
series = df["ConvertedCompYearly"].agg(["median", "mean"])
display(series)


median     67845.000000
mean      170761.277849
Name: ConvertedCompYearly, dtype: float64

---
## More Examples on Exploring Data


---

---
### E1. How many developers from Japan knows Python?
We can create a filter for both the conditions (Japan, and Python), and count how the True values from the filter either by using previously `value_counts()` or by the `sum()` method.  

`sum()` is normally used on numerical values, but it can also work on Booleans; it returns the count of True values.

---

In [16]:
# Filter respondents who is in Japan
filt_country = df["Country"] == "Japan"

# Filter for respondents that works with Python
# This will result to a filter of respondents who works in Japan AND works with Python
filt = df.loc[filt_country]["LanguageHaveWorkedWith"].str.contains("Python").dropna()
# dropna() drops NaN values because some respondents did not fill out the question.

---
#### E1. Python users by country
Using a groupby's `apply()` method, let us determine the proportion of developers by country who knows Python.  

The `groupby.apply()` method takes a function name as it's argument and that function should be able to take a Series as its argument and return a DataFrame. `apply()` will take care of combining the results back together into a single DataFrame or Series.

---

In [17]:
# First we create a DataFrame.groupby object:
country_group = df.groupby("Country")


# Define function for getting proportion of devs who knows Python
def python_proportion(series):
    # This only compares the number of devs who know Python
    # to the total non-NA values. Use len(series) to include
    # NA values.
    n_python = series.str.contains("Python").sum()
    return (n_python) / series.count()


series_python_proportions = country_group["LanguageHaveWorkedWith"].apply(
    python_proportion
)
display(series_python_proportions)

Country
Afghanistan                             0.390625
Albania                                 0.339286
Algeria                                 0.527778
Andorra                                 0.266667
Angola                                  0.272727
                                          ...   
Venezuela, Bolivarian Republic of...    0.440678
Viet Nam                                0.447368
Yemen                                   0.285714
Zambia                                  0.352941
Zimbabwe                                0.392857
Name: LanguageHaveWorkedWith, Length: 180, dtype: float64

In [18]:
# By using a lambda fraction, we can achieve the same results:
series_python_proportions_2 = country_group["LanguageHaveWorkedWith"].apply(
    lambda x: x.str.contains("Python").sum() / x.count()
)
display(series_python_proportions_2)

Country
Afghanistan                             0.390625
Albania                                 0.339286
Algeria                                 0.527778
Andorra                                 0.266667
Angola                                  0.272727
                                          ...   
Venezuela, Bolivarian Republic of...    0.440678
Viet Nam                                0.447368
Yemen                                   0.285714
Zambia                                  0.352941
Zimbabwe                                0.392857
Name: LanguageHaveWorkedWith, Length: 180, dtype: float64

---
Create a df by concatenating the Series of people who knows Python (in proportion and in frequency), and the total respondents by country (non-NA). 

---

In [19]:
# Series
series_python_freq = country_group["LanguageHaveWorkedWith"].apply(
    lambda x: x.str.contains("Python").sum()
)

series_n_respondents = country_group["LanguageHaveWorkedWith"].count()

# Create df
df_python = pd.concat(
    objs=[series_python_proportions, series_python_freq, series_n_respondents],
    keys=["PercentPython", "KnowsPython", "TotalRespondents"],
    axis="columns",
)

display(df_python)

,PercentPython,KnowsPython,TotalRespondents
Country,,,
Afghanistan,0.390625,25,64
Albania,0.339286,19,56
Algeria,0.527778,19,36
Andorra,0.266667,4,15
Angola,0.272727,3,11
...,...,...,...
"Venezuela, Bolivarian Republic of...",0.440678,52,118
Viet Nam,0.447368,136,304
Yemen,0.285714,4,14


---
There are numerous ways to achieve the df we got from the previous example, another one is instead of creating 3 Series independently, we can create the `series_python_porportions` as a derived Series from the other 2 Series.

The reason why the previous example was concatenated that way was for convenience since `series_python_proportions` was already created from a previous example.

---

In [20]:
# Instead of using the apply function, just derive the series_python_proportions
# Series from the two existing Series.
series_python_proportions_2 = series_python_freq / series_n_respondents
df_python2 = pd.concat(
    objs=[series_python_proportions_2, series_python_freq, series_n_respondents],
    keys=["PercentPython", "KnowsPython", "TotalRespondents"],
    axis="columns",
)

display(df_python2)

,PercentPython,KnowsPython,TotalRespondents
Country,,,
Afghanistan,0.390625,25,64
Albania,0.339286,19,56
Algeria,0.527778,19,36
Andorra,0.266667,4,15
Angola,0.272727,3,11
...,...,...,...
"Venezuela, Bolivarian Republic of...",0.440678,52,118
Viet Nam,0.447368,136,304
Yemen,0.285714,4,14
